# Set up GRIT data for 12lbs(ablation and localization only)

## Useful Links

CLIP paper:
https://arxiv.org/pdf/2103.00020.pdf

CLIP Github repository:
https://github.com/openai/CLIP

CLIP Variants:
https://huggingface.co/mlunar/clip-variants

GRIT paper:
https://arxiv.org/pdf/2204.13653.pdf

GRIT official website:
https://allenai.org/project/grit/data

GRIT Github repo:
https://github.com/allenai/grit_official

Object detection variant 1(not working)
https://www.pinecone.io/learn/zero-shot-object-detection-clip/

Object detection variant 2(adapting-CLIP)
https://github.com/pals-ttic/adapting-CLIP

In [ ]:
######## Mount the drive ########

from google.colab import drive
drive.mount('/content/drive/')

######## DIRPATH ########
DIRPATH_GRIT = '/content/drive/MyDrive/grit_official/'
DIRPATH_DATA = os.path.join(DIRPATH_GRIT, 'data/downloaded_logs/default_job/data/downloaded/GRIT/')
DIRPATH_LOCAL = '/content/GRIT-LOCAL/'

Mounted at /content/drive/


In [ ]:
######## Clone the repo if not ########
if(os.path.exists(DIRPATH_GRIT)):
    print('grit_official already exists')
else:
    !git clone https://github.com/allenai/grit_official.git $DIRPATH_GRIT

grit_official already exists


In [ ]:
######## Install dependencies ########
%cd $DIRPATH_GRIT
!pip install -r requirements.txt

/content/drive/MyDrive/grit_official
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 71.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 74.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 

In [ ]:
######## Import dependencies ########
import os
import numpy as np
import h5py
from omegaconf import DictConfig, OmegaConf
from tqdm import tqdm
import subprocess
from zipfile import ZipFile
from PIL import Image
import fiftyone.zoo as foz
from utils.io import (list_dir, mkdir_if_not_exists, extract_zip, extract_targz,
    download_from_url, load_json_object)
import shutil

In [ ]:
######## Download the data ########
if(os.path.exists(DIRPATH_DATA)):
    print('data already exists')
else:
    %cd $DIRPATH_GRIT
    !bash download.sh
    !python -m generate_distortions

data already exists


## Check required images existing

In [ ]:
# Chekc the images in the json file
with open(os.path.join(DIRPATH_DATA,'samples/ablation/localization.json')) as f:
    data = json.load(f)

    # Example of the json element
    # [
    #     {
    #         "example_id": "coco_loc_test-reserve_spoon_527067",
    #         "image_id": "coco/test2015/COCO_test2015_000000527067.jpg",
    #         "output_options": null,
    #         "task_bbox": null,
    #         "task_name": "localization",
    #         "task_query": "spoon"
    #     },
    #     ... 
    # ]

    ######### Don't do this because it is too slow #########
    
    # Check the images
    # for i in range(10):
    #     print(data[i]['image_id'])
    #     print(os.path.join("/content/",data[i]['image_id']))
    #     print( os.path.exists(os.path.join("/content/",data[i]['image_id'])))

In [ ]:
# Check the number of the images
print(len(data))

# Check if the images are in the folder using vectorization
def checkfile(di):
  return os.path.exists(os.path.join(DIRPATH_LOCAL,'images',di['image_id']))
cnt = len(list(filter(checkfile,data)))
print("ALL THE IMAGES EXISTING - ",cnt==len(data),cnt,len(data))

# Check the types of the images need to be downloaded
folderset=set({})
for di in data:
  folderset.add("/".join(di['image_id'].split('/')[:-1]))
  # check if the image exists
folderset

21078
ALL THE IMAGES EXISTING -  True


{'coco/test2015',
 'distorted/localization/coco/test2015',
 'nyuv2',
 'open_images/test'}

In [ ]:
# Check the number of the images in the individual folders
# !ls -1U $DIRPATH_DATA/images/distorted/localization/coco/test2015 | wc -l

381


## coco/test2015

In [ ]:
# Copy coco/test2015 to local
COPY_FILEPATH = 'images/coco/test2015.zip'
COPY_DIRPATH = os.path.join(DIRPATH_LOCAL,COPY_FILEPATH.split('.')[0]+'/')
# Unpack
if(not os.path.exists(COPY_DIRPATH)):
    os.makedirs(COPY_DIRPATH)    
with ZipFile(os.path.join(DIRPATH_DATA,COPY_FILEPATH),'r') as f:
    f.extractall(COPY_DIRPATH)

## distorted/localization/coco/test2015

In [ ]:
# distorted images are generated by the distortion masks stored in a hdf5 file
# Unpacked hdf5 files are listed as $DIRPATH_DATA/distortions/dist_deltas_{task}_{subset}.hdf5
# The hdf5 files are packed in $DIRPATH_DATA/distortions.tar.gz

# Modification of generate_distortions.py
subset = 'ablation'
task = 'localization'
filename = f'dist_deltas_{task}_{subset}.hdf5'
grit_paths = DIRPATH_LOCAL # This is target directory for output images and source directory for input images (undistorted) but not mask images

# Load the samples from the json file and filter the images to be distorted
samples = load_json_object(os.path.join(DIRPATH_DATA,f'samples/{subset}/{task}.json'))
samples = [s for s in samples if 'distorted' in s['image_id']]

# Load the hdf5 file
deltas = h5py.File(os.path.join(DIRPATH_DATA,'distortions', filename),'r')
for sample in samples:
    # Get the image id and delta(mask)
    example_id = sample['example_id']
    delta = deltas[example_id[:-5]][()]
    dist_image_id = sample['image_id']
    undist_image_id = '/'.join(dist_image_id.split('/')[2:])

    # Load the image and distort it
    img = Image.open(os.path.join(
        grit_paths,
        'images',
        undist_image_id)).convert('RGB')
    dist_img = np.asarray(img) + delta
    dist_img = Image.fromarray(dist_img.astype(np.uint8))
    dist_img_path = os.path.join(
        grit_paths,
        'images',
        dist_image_id)
    
    # Save the distorted image
    mkdir_if_not_exists(
        os.path.dirname(dist_img_path),
        recursive=True)
    dist_img.save(dist_img_path)

## nyuv2

In [ ]:
# Set the image directory
img_dir_local = DIRPATH_LOCAL+'images/nyuv2'
mkdir_if_not_exists(img_dir_local,recursive=True)
img_dir_drive = DIRPATH_DATA+'images/nyuv2'

In [ ]:
#download_from_url("http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labeled.mat", img_dir)
# Load the mat file and extract the images
f = h5py.File(os.path.join(img_dir_drive,'nyu_depth_v2_labeled.mat'),'r')
images = f['images'][()].transpose(0,3,2,1)
num_images = images.shape[0]
# Save the images in the right format
for i in tqdm(range(num_images)):
    im = Image.fromarray(images[i])
    im.save(f'{img_dir_local}/{i+1}.jpg')

100%|██████████| 1449/1449 [00:36<00:00, 40.10it/s]


## open_images/test

In [ ]:
# Modification of download/images.py
img_dir = DIRPATH_LOCAL+'images/open_images'
mkdir_if_not_exists(img_dir,recursive=True)

# Get the image ids need to be downloaded from the json files
image_ids = set()
for task in ['localization']:#,'categorization','segmentation']:
    grit_paths = os.path.join(DIRPATH_DATA,'samples/')
    for subset in ['ablation','test']:
        filename = os.path.join(grit_paths, subset, task + '.json')
        samples = load_json_object(filename)
        image_ids.update(
            [os.path.splitext(os.path.basename(s['image_id']))[0] \
                for s in samples if 'open_images' in s['image_id']])

# Download the images
foz.download_zoo_dataset(
    "open-images-v6",
    dataset_dir=img_dir,
    split='test',
    image_ids=image_ids)

 100% |█████████████| 24259/24259 [48.6m elapsed, 0s remaining, 7.7 samples/s]      
Dataset info written to '/content/GRIT-LOCAL/images/open_images/info.json'


(<fiftyone.zoo.datasets.ZooDatasetInfo at 0x7f4a04d576a0>,
 '/content/GRIT-LOCAL/images/open_images')

In [ ]:
# Move the images to the right directory
src_dir = f'{img_dir}/test/data'
tgt_dir = f'{img_dir}/test'
allfiles = os.listdir(src_dir)
for f in allfiles:
    srcfile = os.path.join(src_dir,f)
    tgtfile = os.path.join(tgt_dir,f)
    shutil.move(srcfile, tgtfile)

# Remove the unnecessary files
os.rmdir(src_dir)
subprocess.call(
    f'rm -rf {img_dir}/test/data & rm -rf {img_dir}/test/metadata & rm -rf {img_dir}/test/labels',
    shell=True)

0

## tar gzip all neccessary images

This is required because the images are too many to be worked with Google Drive.

The images are stored in a tar.gz file and then uploaded to the Google Drive folder.

Thus, when evertime start working with a Google Colab file, the images are required to be loaded to local storage from the tar.gz file in Google Drive.

In [ ]:
%cd /content/
!tar -czf grit_local.tar.gz GRIT-LOCAL

/content


In [82]:
!cp grit_local.tar.gz /content/drive/MyDrive/COLAB-FILES/12lbs/
!cp $DIRPATH_DATA/samples/ablation/localization.json /content/drive/MyDrive/COLAB-FILES/12lbs/ablation-localization.json